In [15]:
import json
import pandas as pd
import psycopg2 as pg
from sqlalchemy import create_engine
from zipfile import ZipFile
from datetime import date, datetime

In [16]:
data = pd.read_json('../sql/schemas/user_address.json')
data

,column_name,column_type,is_primary_key,is_unique,is_null_able
0,first_name,varchar(256),False,False,NOT NULL
1,last_name,varchar(256),False,False,NOT NULL
2,email,varchar(256),False,False,NOT NULL
3,address,varchar(256),False,False,NOT NULL
4,created_at,timestamp,False,False,NOT NULL


In [17]:
data.drop(['is_primary_key', 'is_unique'], axis=1, inplace=True)

In [18]:
schema_list = []
for i, row in data.iterrows():
    row = tuple(row.astype("string"))
    row_str = ' '.join(list(row))
    schema_list.append(row_str)

query = """
CREATE TABLE if not exists user_address (
"""
for i, schema in enumerate(schema_list):
    query += schema
    if i < len(schema_list) - 1:
        query += ','
query += ')'
query

'\nCREATE TABLE if not exists user_address (\nfirst_name varchar(256) NOT NULL,last_name varchar(256) NOT NULL,email varchar(256) NOT NULL,address varchar(256) NOT NULL,created_at timestamp NOT NULL)'

In [19]:
#Init Posgres conn
conn = pg.connect(database='shipping_orders',
                  user='postgres',
                  password='Sukses37',
                  host='localhost',
                  port='5432')

conn.autocommit=True
cursor=conn.cursor()

In [20]:
# Execute DDL
try:
    cursor.execute(query)
except Exception as e:
    print(e)

In [8]:
# read zip file
zf = ZipFile('../temp/dataset-small.zip')
df = pd.read_csv(
    zf.open('dataset-small.csv'), 
    header=None, names=data.column_name.tolist(), 
    parse_dates=['created_at'])
df.created_at = pd.to_datetime(df.created_at).dt.tz_localize(None)
df.head()

,first_name,last_name,email,address,created_at
0,Cawuk,Tarihoran,sabriaryani@perum.edu,"Jalan Jakarta No. 7Semarang, Nusa Tenggara Ti...",2018-08-10 20:48:35.506
1,Darmaji,Januar,jwahyudin@yahoo.com,"Gang Soekarno Hatta No. 27Payakumbuh, Maluku U...",2018-06-17 19:31:04.314
2,Bahuraksa,Mansur,zulfaharyanti@pd.sch.id,"Jl. Raya Ujungberung No. 2Palembang, Maluku Ut...",2018-10-28 11:20:50.360
3,Among,Simanjuntak,kusuma67@perum.id,"Jalan Cikutra Timur No. 03Tanjungpinang, Papua...",2018-10-26 00:47:32.831
4,Calista,Najmudin,gastiutama@gmail.com,Jalan Asia Afrika No. 589Kota Administrasi Jak...,2018-12-14 17:09:49.514


In [9]:
# filter data created at '2018-02-01' until '2018-12-31'
df_filtered = df[(df.created_at >= datetime(2018,2,1)) & (df.created_at <= datetime(2018,12,31))]
df_filtered.head()

,first_name,last_name,email,address,created_at
0,Cawuk,Tarihoran,sabriaryani@perum.edu,"Jalan Jakarta No. 7Semarang, Nusa Tenggara Ti...",2018-08-10 20:48:35.506
1,Darmaji,Januar,jwahyudin@yahoo.com,"Gang Soekarno Hatta No. 27Payakumbuh, Maluku U...",2018-06-17 19:31:04.314
2,Bahuraksa,Mansur,zulfaharyanti@pd.sch.id,"Jl. Raya Ujungberung No. 2Palembang, Maluku Ut...",2018-10-28 11:20:50.360
3,Among,Simanjuntak,kusuma67@perum.id,"Jalan Cikutra Timur No. 03Tanjungpinang, Papua...",2018-10-26 00:47:32.831
4,Calista,Najmudin,gastiutama@gmail.com,Jalan Asia Afrika No. 589Kota Administrasi Jak...,2018-12-14 17:09:49.514


In [14]:
cols = ",".join([str(i) for i in df_filtered.columns.tolist()])
try:
    for i, row in df_filtered.iterrows():
        sql = "INSERT INTO user_address (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
        cursor.execute(sql, tuple(row))

except Exception as e:
    print(e)